In [7]:
import torch    
import json 
from ptlpinns.models import model

### Load model

In [8]:
with open('/home/dda24/PTL-PINNs/ptlpinns/models/train/undamped_k12/training_log.json', 'r') as f:
    training_log = json.load(f)

k = training_log['k']
hidden_layers = training_log['hidden_layers']
n_frequencies = training_log['n_frequencies']
bias = training_log['bias']
use_sine = training_log['use_sine']
use_fourier = training_log['use_fourier']
scale = training_log['scale']

In [9]:
pinn = model.Multihead_model_fourier(k=k, bias=bias, use_sine=use_sine, use_fourier=use_fourier, scale=scale, n_frequencies=n_frequencies, HIDDEN_LAYERS=hidden_layers)
pinn.load_state_dict(torch.load('/home/dda24/PTL-PINNs/ptlpinns/models/train/undamped_k12/model_undamped_k12.pth'))

<All keys matched successfully>

### Compute H Dict

### Frequency